RNA het pipe

working on getting containerized freebayes working now

In [2]:
%%bash
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG2.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG3.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG4.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG5.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG6.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG7.fa.gz
gunzip *LG*.fa.gz
cat *LG*.fa > redclover_ref.fa
#gff:
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/gff3/trifolium_pratense/Trifolium_pratense.Trpr.38.chr.gff3.gz

--2018-05-09 16:17:44--  ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz
           => ‘Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz’
Resolving ftp.ensemblgenomes.org (ftp.ensemblgenomes.org)... 193.62.197.94
Connecting to ftp.ensemblgenomes.org (ftp.ensemblgenomes.org)|193.62.197.94|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/plants/release-38/fasta/trifolium_pratense/dna ... done.
==> SIZE Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz ... 7617692
==> PASV ... done.    ==> RETR Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz ... done.
Length: 7617692 (7.3M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  155K 48s
    50K .......... .......... .......... .......... ..........  1%  307K 36s
   100K .......... .......... .......... .......... ..........  2%  298K 32s
   1

Make an extreme example dataset

In [ ]:
touch all.fastq
declare -a arr=("ERR1665298"
"ERR1665299"
"ERR1665300"
"ERR1665301"
"ERR1665302"
"ERR1665303"
"ERR1665304"
"ERR1665305"
"ERR1665306"
"ERR1665307"
"ERR1665308")

for i in "${arr[@]}"
do
	docker run --rm -v "$(pwd)":/data -w /data inutano/sra-toolkit fastq-dump "$i"
	cat "$i".fastq >> all.fastq
    rm -f "$i".fastq
done

In [24]:
%%bash
cat Trifolium_pratense.Trpr.dna.chromosome.LG1.fa | wc -l

469035


In [25]:
%%bash
head -n 234500 Trifolium_pratense.Trpr.dna.chromosome.LG1.fa > chrom0.5.fa


In [ ]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/bwa bwa index chrom0.5.fa

In [33]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/bwa bwa mem chrom0.5.fa all.fastq > temp

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 196080 sequences (10000080 bp)...
[M::process] read 196080 sequences (10000080 bp)...
[M::mem_process_seqs] Processed 196080 reads in 5.634 CPU sec, 5.503 real sec
[M::process] read 196080 sequences (10000080 bp)...
[M::mem_process_seqs] Processed 196080 reads in 5.618 CPU sec, 5.377 real sec
[M::process] read 196080 sequences (10000080 bp)...
[M::mem_process_seqs] Processed 196080 reads in 6.882 CPU sec, 6.682 real sec
[M::process] read 196080 sequences (10000080 bp)...
[M::mem_process_seqs] Processed 196080 reads in 7.174 CPU sec, 6.907 real sec
[M::process] read 196080 sequences (10000080 bp)...
[M::mem_process_seqs] Processed 196080 reads in 5.996 CPU sec, 5.769 real sec
[M::process] read 196080 sequences (10000080 bp)...
[M::mem_process_seqs] Processed 196080 reads in 7.303 CPU sec, 7.098 real sec
[M::process] read 196080 sequences (10000080 bp)...
[M::mem_process_seqs] Processed 196080 reads in 6.184 CPU sec, 5.952 

In [34]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools samtools view -bS temp > temp1

In [35]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools samtools view -b -F 4 temp1 > mappedchrom0.5.bam

In [ ]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools:1.2 samtools bam2fq -O -s mappedchrom0.5.fastq mappedchrom0.5.bam 

In [43]:
%%bash
cat mappedchrom0.5.fastq ERR1665297.fastq > highhet_chrom0.5.fastq

now we have reads representing high heterozygosity for half of chromosome 1, and the rest of the genome represented by a single individual's reads.  This can be used to test our pipeline

In [42]:
%%bash
fastqp highhet_chrom0.5.fastq

highhet_chrom0.5.fastq	reads	None	20856498
highhet_chrom0.5.fastq	read_len	51	1604346
highhet_chrom0.5.fastq	q05	1	26
highhet_chrom0.5.fastq	q25	1	30
highhet_chrom0.5.fastq	q50	1	31
highhet_chrom0.5.fastq	q75	1	31
highhet_chrom0.5.fastq	q95	1	34
highhet_chrom0.5.fastq	q05	2	28
highhet_chrom0.5.fastq	q25	2	31
highhet_chrom0.5.fastq	q50	2	31
highhet_chrom0.5.fastq	q75	2	33
highhet_chrom0.5.fastq	q95	2	34
highhet_chrom0.5.fastq	q05	3	26
highhet_chrom0.5.fastq	q25	3	31
highhet_chrom0.5.fastq	q50	3	31
highhet_chrom0.5.fastq	q75	3	34
highhet_chrom0.5.fastq	q95	3	34
highhet_chrom0.5.fastq	q05	4	28
highhet_chrom0.5.fastq	q25	4	35
highhet_chrom0.5.fastq	q50	4	35
highhet_chrom0.5.fastq	q75	4	37
highhet_chrom0.5.fastq	q95	4	37
highhet_chrom0.5.fastq	q05	5	26
highhet_chrom0.5.fastq	q25	5	35
highhet_chrom0.5.fastq	q50	5	35
highhet_chrom0.5.fastq	q75	5	37
highhet_chrom0.5.fastq	q95	5	37
highhet_chrom0.5.fastq	q05	6	32
highhet_chrom0.5.fastq	q25	6	35
highhet_chrom0.5.fastq	q50	6	35
highhet_chrom0.5.f

At 188 bytes per read of 51 length we estimate 27,459,402 reads in input file.
Bin size (-s) set to 13.
Approximately 10% complete at read 2,745,951 in 00:00:14
Approximately 20% complete at read 5,491,889 in 00:00:29
Approximately 30% complete at read 8,237,827 in 00:00:44
Approximately 40% complete at read 10,983,765 in 00:00:59
Approximately 50% complete at read 13,729,703 in 00:01:15
Approximately 60% complete at read 16,475,654 in 00:01:30
Approximately 70% complete at read 19,221,592 in 00:01:45
/home/cvisger/anaconda3/lib/python3.6/site-packages/matplotlib/artist.py:896: MatplotlibDeprecationWarning: The set_axis_bgcolor function was deprecated in version 2.0. Use set_facecolor instead.
  return func(v)
There were 20,856,498 reads in the file. Analysis finished in 00:02:11.


In [44]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data fjukstad/trimmomatic  SE highhet_chrom0.5.fastq highhet_chrom0.5.trimmed.fq SLIDINGWINDOW:4:30
        

TrimmomaticSE: Started with arguments:
 highhet_chrom0.5.fastq highhet_chrom0.5.trimmed.fq SLIDINGWINDOW:4:30
Automatically using 1 threads
Quality encoding detected as phred33
Input Reads: 34951140 Surviving: 30027837 (85.91%) Dropped: 4923303 (14.09%)
TrimmomaticSE: Completed successfully


In [45]:
%%bash
fastqp highhet_chrom0.5.trimmed.fq

highhet_chrom0.5.trimmed.fq	reads	None	30027840
highhet_chrom0.5.trimmed.fq	read_len	13	11525
highhet_chrom0.5.trimmed.fq	read_len	42	9795
highhet_chrom0.5.trimmed.fq	read_len	15	12324
highhet_chrom0.5.trimmed.fq	read_len	51	733730
highhet_chrom0.5.trimmed.fq	read_len	47	8419
highhet_chrom0.5.trimmed.fq	read_len	50	30038
highhet_chrom0.5.trimmed.fq	read_len	6	12410
highhet_chrom0.5.trimmed.fq	read_len	11	10770
highhet_chrom0.5.trimmed.fq	read_len	14	13305
highhet_chrom0.5.trimmed.fq	read_len	43	8349
highhet_chrom0.5.trimmed.fq	read_len	19	11390
highhet_chrom0.5.trimmed.fq	read_len	7	13303
highhet_chrom0.5.trimmed.fq	read_len	44	7517
highhet_chrom0.5.trimmed.fq	read_len	39	10721
highhet_chrom0.5.trimmed.fq	read_len	41	9717
highhet_chrom0.5.trimmed.fq	read_len	20	10600
highhet_chrom0.5.trimmed.fq	read_len	33	8557
highhet_chrom0.5.trimmed.fq	read_len	35	9700
highhet_chrom0.5.trimmed.fq	read_len	12	12895
highhet_chrom0.5.trimmed.fq	read_len	22	11475
highhet_chrom0.5.trimmed.fq	read_len	21	

At 108 bytes per read of 43 length we estimate 49,220,203 reads in input file.
Bin size (-s) set to 24.
Approximately 10% complete at read 4,922,040 in 00:00:14
Approximately 20% complete at read 9,844,056 in 00:00:29
Approximately 30% complete at read 14,766,072 in 00:00:44
Approximately 40% complete at read 19,688,088 in 00:00:59
Approximately 50% complete at read 24,610,104 in 00:01:14
Approximately 60% complete at read 29,532,144 in 00:01:29
/home/cvisger/anaconda3/lib/python3.6/site-packages/matplotlib/artist.py:896: MatplotlibDeprecationWarning: The set_axis_bgcolor function was deprecated in version 2.0. Use set_facecolor instead.
  return func(v)
There were 30,027,840 reads in the file. Analysis finished in 00:01:39.


In [ ]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data ebio/gsnap gmap_build -D . -d redclover_ref --nthreads 24 -k 15 redclover_ref.fa


In [ ]:
%%bash
 docker run --rm -v "$(pwd)":/data -w /data ebio/gsnap gsnap -D ./redclover_ref -d redclover_ref --nthreads 24 --novelsplicing 1 --format sam --read-group-platform=illumina --force-single-end highhet_chrom0.5.trimmed.fq > temp2


In [ ]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools samtools view -Sbh temp2 > temp3


In [ ]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools samtools sort -O bam -T 12345 temp3 > highhet_chrom0.5.sorted.bam

In [50]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools:1.2 samtools index highhet_chrom0.5.sorted.bam

In [51]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools:1.2 samtools stats highhet_chrom0.5.sorted.bam > highhet_chrom0.5.sorted.stats.txt

In [52]:
%%bash
#freebayes container seems to crash repeatly at the same point -- using clean install of freebayes instead
#docker run --rm -v "$(pwd)":/data -w /data maxulysse/freebayes freebayes --min-alternate-fraction 0.1 --ploidy 4 --hwe-priors-off --allele-balance-priors-off --max-complex-gap 50 -f redclover_ref.fa highhet_chrom0.5.sorted.bam > highhet_chrom0.5_to_redclover_ref.vcf
~/github/freebayes/bin/freebayes --min-alternate-fraction 0.1 --ploidy 4 --hwe-priors-off --allele-balance-priors-off --max-complex-gap 50 -f redclover_ref.fa highhet_chrom0.5.sorted.bam > highhet_chrom0.5_to_redclover_ref.vcf


Unable to find image 'maxulysse/freebayes:latest' locally
latest: Pulling from maxulysse/freebayes
85b1f47fba49: Pulling fs layer
aa1fe734d5e8: Pulling fs layer
c688e000dcf9: Pulling fs layer
c688e000dcf9: Verifying Checksum
c688e000dcf9: Download complete
85b1f47fba49: Verifying Checksum
85b1f47fba49: Download complete
85b1f47fba49: Pull complete
aa1fe734d5e8: Verifying Checksum
aa1fe734d5e8: Download complete
aa1fe734d5e8: Pull complete
c688e000dcf9: Pull complete
Digest: sha256:cd15220257dcd20a73c78c2bc3301cf931d3a8976a81d7534e6c899bb9f2b7f5
Status: Downloaded newer image for maxulysse/freebayes:latest
index file redclover_ref.fa.fai not found, generating...
*** Error in `freebayes': free(): invalid next size (normal): 0x00000000022a3bc0 ***
Error: signal 11:
freebayes[0x4ab795]
/lib/x86_64-linux-gnu/libc.so.6(+0x350e0)[0x7fc677ce60e0]
/lib/x86_64-linux-gnu/libc.so.6(abort+0x232)[0x7fc677ce7532]
/lib/x86_64-linux-gnu/libc.so.6(+0x731b4)[0x7fc677d241b4]
/lib/x86_64-linux-gnu/libc.so.